In [ ]:
from selenium import webdriver
import time
import random
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
#desired Capabilities클래스는 테스트를 실행할 수 있는 브라우저/환경의 동작을 정의하는 환경을 설정하는 데 도움이 됩니다.
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

# Audible 장르별 url

In [ ]:
def get_category_numbering(list):
    rtn_list = []
    for item in list:
        rtn_list.append( item.get_text())
    return rtn_list

url = 'https://www.audible.com//most-listened-audiobooks'
request = urllib.request.urlopen(url)
html = request.read()
soup = BeautifulSoup(html, 'html.parser')

#장르별카테고리 링크 가져오기
find_ul = soup.find('ul', class_='bc-list bc-spacing-medium bc-list-nostyle')

link=[]
for href in find_ul.find_all('li'):
    a = href.find_all('a')
    for i in a:
        b = i['href']
        link.append('https://www.audible.com/'+b)
        
# 카테고리 리스트 번호 매김 
li_list = find_ul.find_all('li')
select_list = get_category_numbering(li_list)     
select_list =  select_list[1::] 

# 수집한 리스트를 DataFrame으로 전환
data = {'link' : b,
        'genre' : select_list}

df = pd.DataFrame(data)
df

In [ ]:
link

# driver 선언

In [ ]:
book_selector='#product-list-a11y-skiplink-target > span > ul > div > li:nth-child(17) > div > div.bc-col-responsive.bc-spacing-top-none.bc-col-8 > div > div.bc-col-responsive.bc-col-5 > div > div:nth-child(1) > div > a > div > img'
genre_num ='18581095011'

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

for url in link:
    
    if genre_num in url: 
    
        driver.get(url)
        time.sleep(1.1)
        
        # 책 버튼 클릭
        driver.find_element_by_css_selector(book_selector).click()
        time.sleep(5)

# SCROLL

In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight")  #스크롤의 위치를 찾습니다.                

while True:
    
    for i in range(4):  #스크롤을 내리면 3번까지는 자동적으로 댓글 갱신이 되는데, 4번째에는 "더보기" 버튼이 활성화됩니다.                                                                    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤을 가능한 아래까지 내립니다.
        time.sleep(random.uniform(1,1.7))  
    
    try:
        driver.find_element_by_xpath('//div/span/button[@aria-label="Show More Reviews"]').click()

        time.sleep(random.uniform(1,1.7))
    except:  #같은 조건에서 "더보기" 버튼이 활성화 되지 않으면 이 과정을 무시합니다.
        pass
        
    new_height = driver.execute_script("return document.body.scrollHeight")  
    #스크롤을 가능한 아래까지 내렸을 때, 댓글이 갱신되어 스크롤의 위치와 크기가 바뀌면, 다시 그 스크롤을 최대한 아래까지 내립니다.
    if new_height == last_height:   
        #스크롤을 가능한 아래까지 내렸을 때, 댓글이 갱신되지 않아 스크롤의 위치와 크기가 바뀌지 않는다면, 이 루프를 끝냅니다.
        break
    last_height = new_height

# 장르별 리뷰와 별점 크롤링
- us/uk 리뷰가 각각 다른 페이지에 존재함
- 메타데이터 제목에 정리!

## US 리뷰

In [ ]:
soup = bs(driver.page_source, 'lxml')

review_Title = []
review_Date= []
review_content=[]

N = int(5000)

for w in range(N):
    
    #xpath 4번째 슬러시가 4=uk //3=us
    
    #TITLE
    title = driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[3]/div[2]/div[{}]/div[2]/h4'.format(w))
    for i in title:
        
        com = i.text
        review_Title.append([com])

    #DATE
    Date = driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[3]/div[2]/div[{}]/div[1]/div/div[2]/span/ul/li[2]'.format(w))
    for j in Date:
        d = j.text
        review_Date.append([d])
        
    #CONTENT
    content = driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[3]/div[2]/div[{}]/div[2]/p'.format(w))
    if len(content) == 0:
        pass
        
    elif (len(content) > 0) and (len(content) <= 2):
        review_content.append([content[0].text])
    
    else:
        insert = []
        for i in content:
            insert.append(i.text)
        review_content.append(insert[1::2])
print(len(review_Title), len(review_Date) , len(review_content))  

#STAR(Overall만 불러와짐..ㅎㅎ)
star=[]

for w in range(len(review_Title)+1): 
    star1=driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[3]/div[2]/div[{}]/div[1]/span/ul/li[1] | //*[@id="center-15"]/div/div[3]/div/div[4]/div[2]/div[{}]/div[1]/span/ul/li'.format(w,w))
    for i in star1:
        star.append(i.text)
        
star=[x for x in star if x] # 빈칸 제거
print(len(star))

In [ ]:
star00=[]
for n in range(len(star)):
    i = star[n]
    i = i.split('\n')
    star00.append([i[1]])
    
    
star_US=[]    
for j in star00:
    z = re.findall(r'\d', str(j))
    star_US.append([z[0]])
print(len(star_US))

In [ ]:
#크롤링 된 데이터 프레임으로 만들기
data = {'Title' : review_Title,
        'Review' : review_content,
       'Date':review_Date}
df = pd.DataFrame(data)
star=pd.DataFrame(star_US)
result=pd.concat([df,star], axis=1)
result1=result.copy()
result1['Star']=result1[0]
del result1[0]
result1

In [ ]:
result1.to_csv('Travel & Tourism#17.csv')

In [ ]:
driver.quit()

# UK

# 드라이버 선언

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

for url in link:
    if genre_num in url: 
    
        driver.get(url)
        time.sleep(1.1)
        
        # 책 버튼 클릭
        driver.find_element_by_css_selector(book_selector).click()
        time.sleep(5)
        #UK review buttom 클릭
        driver.find_element_by_css_selector('#UK > div > h2').click()
        time.sleep(5)

In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight")  #스크롤의 위치를 찾습니다.                

while True:
    
    for i in range(3):  #스크롤을 내리면 3번까지는 자동적으로 댓글 갱신이 되는데, 4번째에는 "더보기" 버튼이 활성화됩니다.                                                                    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤을 가능한 아래까지 내립니다.
        time.sleep(random.uniform(1,1.7))  
    
    try:
        driver.find_element_by_xpath('//*[@id="pagingButtonsUK"]/div/span/button').click()
        time.sleep(random.uniform(1,1.7))
    except:  #같은 조건에서 "더보기" 버튼이 활성화 되지 않으면 이 과정을 무시합니다.
        pass
        
    new_height = driver.execute_script("return document.body.scrollHeight")  
    #스크롤을 가능한 아래까지 내렸을 때, 댓글이 갱신되어 스크롤의 위치와 크기가 바뀌면, 다시 그 스크롤을 최대한 아래까지 내립니다.
    if new_height == last_height:   
        #스크롤을 가능한 아래까지 내렸을 때, 댓글이 갱신되지 않아 스크롤의 위치와 크기가 바뀌지 않는다면, 이 루프를 끝냅니다.
        break
    last_height = new_height

In [ ]:
UK_review_Title = []
UK_review_Date= []
UK_review_content=[]

#요고 변경해야함.
#N = int(input('반복횟수를 결정해주세요 :'))
N = int(1000)

for w in range(N):
    
    #xpath 4번째 슬러시가 4=uk //3=us
    #TITLE
    UK_title = driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[4]/div[2]/div[{}]/div[2]/h4'.format(w))
    for i in UK_title:
        com = i.text
        UK_review_Title.append([com])

    #DATE                                   
    UK_Date = driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[4]/div[2]/div[{}]/div[1]/div/div[2]/span/ul/li[2]'.format(w))
    for j in UK_Date:
        d = j.text
        UK_review_Date.append([d])
        
    #CONTENT
    UK_content = driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[4]/div[2]/div[{}]/div[2]/p'.format(w))
    if len(UK_content) == 0:
        pass
        
    elif (len(UK_content) > 0) and (len(UK_content) <= 2):
        UK_review_content.append([UK_content[0].text])
    
    else:
        insert = []
        for i in UK_content:
            insert.append(i.text)
        UK_review_content.append(insert[1::2])
        
print(len(UK_review_Title), len(UK_review_Date) , len(UK_review_content)) 

In [ ]:
#STAR
UK_star=[]

for w in range(N): 
    star1=driver.find_elements_by_xpath('//*[@id="center-15"]/div/div[3]/div/div[4]/div[2]/div[{}]/div[1]/span/ul/li[1] | //*[@id="center-15"]/div/div[3]/div/div[4]/div[2]/div[{}]/div[1]/span/ul/li'.format(w,w))
    
    for i in star1:
        UK_star.append(i.text) 

b=[]
for i in UK_star:
    a=i.split('\n')
    b.append(a)
    

UK_star_dict={"Overall":[], "Performance":[], "Story":[]}

for i, j in b:
    z = re.findall(r'\d', j)
    if 'Overall' in i :
            UK_star_dict['Overall'].append(z[0])
            
    elif 'Performance' in i :
        UK_star_dict['Performance'].append(z[0])
            
    else:
        UK_star_dict['Story'].append(z[0])
    

print(len(UK_star_dict['Overall']), len(UK_star_dict['Performance']), len(UK_star_dict['Story']))                      

In [ ]:
#크롤링 된 데이터 프레임으로 만들기
data1 = {'Title' : UK_review_Title,
        'Review' : UK_review_content,
       'Date':UK_review_Date}
UK_df = pd.DataFrame(data1)
UK_star=pd.DataFrame(UK_star_dict['Overall'])
UK_result=pd.concat([UK_df,UK_star], axis=1)
UK_result1=UK_result.copy()
UK_result1['Star']=UK_result1[0]
del UK_result1[0]
UK_result1

# US+UK data concat

In [ ]:
total= pd.concat([UK_result1,result1], axis=0)
print(len(total))
total

In [ ]:
total.to_csv('Travel & Tourism#17.csv')

In [ ]:
driver.quit()
